<head><b><center>MACHINE LEARNING</center>

<center>Redes Neuronales Artificiales</center>
    

<center>Profesor: Gabriel Jara </center></head><br>
El presente Jupyter Notebook busca:
<ul>
    <li>Retomar conceptos sobre Redes Neuronales Artificiales (ANN).</li>
    <li>Profundizar en el mecanismo de aprendizaje de las ANN</li>
    <li>Analizar la implementación de una red neuronal sin apoyo de librerías especializadas.</li>
</ul>

Fuentes: 
<ul>
    <li>Imágenes han sido robadas de internet y son enlaces a su correspondiente fuente. </li>
</ul> 



<b>En el capítulo anterior </b>

Extendimos a idea del perceptrón básico...
<br> <img src=https://images.deepai.org/glossary-terms/perceptron-6168423.jpg
          width="400"/>

...para construir una red de perceptrones, <b><i>Multi-Layer Perceptron</i></b>, que es nuestra primera Red Neuronal Artificial. 
<br> <img src=https://miro.medium.com/v2/resize:fit:563/1*4_BDTvgB6WoYVXyxO8lDGA.png
          width="350"/>
          
Esta topología será la base sobre la que exploraremos algunas expansiones, ajustándonos a distintos problemas y buscando como mejorar los resultados.           

<b>Implementación Base</b>

La siguiente es la misma implementación que dejamos en la clase previa, con la única diferencia de que hemos comentado el seed para controlar aleatoriedad en la inicialización de pesos. Las Redes Neuronales Artificiales son modelos probabilísticos, no se supone que ante una misma muestra den siempre los mismos resultados, es parte de su valor el que no lo hagan. Mantendremos los seed (pero comentados) por si en algún momento necesitamos resultados reproducibles. 

In [ ]:
import numpy as np

# Clase base para Capa
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError
    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError
        
# Clase para capas densas (fully connected)
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        # YA NO CONTROLAREMOS ALEATORIEDAD
        #np.random.seed(1234)
        self.weights = np.random.rand(input_size, output_size, ) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

# Clase para Capa de Activación. Junto con la capa densa forman perceptrones. 
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error
    
# Clase Red, conecta múltiples capas.
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        input_data = np.array([[x] for x in input_data])
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        x_train = np.array([[x] for x in x_train])
        samples = len(x_train)
        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            
            # Para usar en clasificación (con más de dos clases)
            # calculamos el error promedio entre nodos de salida.
            err = np.mean(err)
            
            # Imprimomos el error promedio de cada época, más que nada
            # para seguimiento del aprendizaje. 
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            
# Funciones de Activación, y su correspondiente función derivada. 
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Función de pérdida y su derivada. 
def mse(y_real, y_hat):
    return (y_real-y_hat)**2

def mse_prime(y_real, y_hat):
    return 2*(y_hat-y_real)

<b>Reconocimiento de Imágenes con MNIST</b>

Ya hemos probado nuestra red con <b><i>Mnist</i></b>, colección de imágenes de dígitos (números) escritos a mano por distintas personas. 
<br> <img src=https://www.ttested.com/gallery/thumbnails/ditch-mnist.jpg
          width="400"/>

La siguiente celda carga muestras para entrenar y para testear resultados.     

In [ ]:
from keras.datasets import mnist
import random
from sklearn.preprocessing import MinMaxScaler

# No necesitamos tantos datos.
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#random.seed(123) # Vamos a controlar la aleatoriedad en adelante. 
X, y = zip(*random.sample(list(zip(X_train, y_train)), 2000))

# Sí necesitamos que la forma de X sea la de un vector, en lugar de una matriz. 
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')
X = np.reshape(X, (X.shape[0], -1))

# Normalizamos Min-Max
X= MinMaxScaler().fit_transform(X)

# Dividimos la muestra en dos, una para entrenar y otra para testing, como tenemos 
# muestra de sobra nos damos el lujo de testear con la misma cantidad que entrenamos.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

# Necesitamos que y_train sea un valor categórico, en lugar de un dígito entero.
y_train_value = y_train # Guardaremos y_train como valor para un observación más abajo.
from keras.utils import to_categorical
y_train = to_categorical(y_train)

A continuación usamos nuestra red neuronal para aprender <i>Mnist</i> y evaluar el resultado.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 16))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(16, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Tal como observamos la clase anterior, el resultado es muy bueno en comparación con lo que se habría logrado por simple azar. Considerando, además, que la muestra de testeo son observaciones no disponibles durante el entrenamiento, podemos decir con alta certeza que el modelo a "aprendido" a reconocer dígitos escritos a mano, incluso si todavía el resultado no es altamente fiable. 

<b>Comentario sobre la inicialización de los pesos $W$</b>

En nuestra implementación cuando se crea una capa densa (<i>fully connected</i>) inicializamos cada peso $w$ con un valor aleatorio. Sin embargo, cuando en la clase anterior implementamos un perceptrón pusimos que los pesos $w$ tuvieran valor cero al inicializarse. Observamos que una vez entrenado el perceptrón los pesos $w$ ajustan su valor, por lo que rapidamene dejan de ser cero. En el caso de una red neuronal, también se produce el ajuste de los pesos $w$ al realizar el entrenamiento. 

<b>¿Por qué entonces no adoptar valor cero para los $w$ al inicializar la red?, ¿qué diferencia hace el que hagamos un trabajo adicional buscando pesos aleatorios cuando podrían simplemente ser cero?</b>

Podemos hacer el experimento de inicializar todos los pesos $w$ en cero, en la red neuronal, y apreciar si ello perjudica la capacidad de aprendizaje del modelo. 
    

In [ ]:
# Sólo cambiaremos la clase FCLayer
# Los pesos w quedarán inicializados en cero. 
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        # Los pesos y el sesgo (b) quedarán todos inicializados en cero.
        # Recuerde que estamos haciendo un experimento, no hemos dicho 
        # que sea buena idea hacer esto... 
        self.weights = np.zeros((input_size, output_size))
        self.bias = np.zeros((1, output_size))

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

A continuación, volveremos a realizar el experimento de entrenar nuestra red para aprender <i>Mnist</i>, veamos si mantiene la capacidad de aprendizaje. 

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 16))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(16, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('\nMATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

Se observa claramente que ahora la red no logra aprender nada, obtiene resultado no mejor que el azar, clasificando todas las observaciones como de la misma clase. ¿Qué clase elige? el resultado es aleatorio, pero debiera haber una marcada tendencia a elegir la clase dominante (o al menos entre las de alta frecuencia) en la muestra de entrenamiento. Claro que la misma clase no tendría por qué tener alta frecuencia en la muestra de testeo.  

In [ ]:
import matplotlib.pyplot as plt

# Crear el histograma
plt.hist(y_train_value, bins=30, edgecolor='black')

# Añadir títulos y etiquetas
plt.title('Histograma de y_train_value')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')

# Mostrar el histograma
plt.show()


El problema que se produce es que, al inicializar todos los pesos en el mismo valor (en este caso cero) y aplicarles el mismo gradiente durante el entrenamiento, el ajuste al que es sometido cada peso es igual en magnitud y dirección ante cualquier muestra, durante todos los epochs. Es decir, los pesos siguen teniendo el mismo valor entre sí por más que se les ajuste, dado que por <b><i>simetría</i></b> en el proceso de ajuste siempre se mantienen sincronizados. El nombre de este problema es, justamente, simetría. 

Cuando dejamos que los perceptrones de nuestra red se entrenen simétricamente le quitamos la capacidad de aprovechar la multiplicidad de capas y perceptrones. En la práctica es como tener una red neuronal donde todas las neuronas son exactamente iguales y siempre se activan al unísono, lo que es equivalente a tener una única neurona. Con algo de suerte sólo puede aspirar a alinearse con la clase dominante, pero no mucho más. 

Así que devolvamos a nuestra red neuronal su capacidad, inicializando pesos aleatorios. 

In [ ]:
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        # YA NO CONTROLAREMOS ALEATORIEDAD
        #np.random.seed(1234)
        self.weights = np.random.rand(input_size, output_size, ) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 16))               
model.add(ActivationLayer(tanh, tanh_prime))
model.add(FCLayer(16, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('\nMATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

<b>Probando otra función de pérdida: <i>Binary Cross Entropy</i></b>

<b>BCE</b> o Entropía Cruzada Binaria, es una función de pérdida más adecuada a la clasificación de lo que es <i>Mean Squared Error</i> (<b>MSE</b>) que venimos usando. 

MSE es una función de pérdida genérica, útil sobre todo en problemas de regresión, pese a que como hemos demostrado también funciona en clasificación. Puede llevar a resultados subóptimos, ya que no maneja bien la naturaleza probabilística de la predicción de clases, en contraste a predecir un valor escalar.

Por otro lado, BCE está diseñada específicamente para tareas de clasificación binaria. Se usa típicamente en redes neuronales cuando la capa de salida utiliza una activación sigmoide (o similar) para generar probabilidades entre 0 y 1, que luego se interpretan como clase. Incluso si el problema de clasificación no es binario, como es el caso de <i>Mnist</i>, sigue siendo adecuado usar BCE dado que la topología de nuestra red tiene tantos nodos de salida como clases posibles, transformando así el resultado de la predicción en una colección de clasificaciones binarias.

La entropía cruzada binaria se define a nivel de muestra de acuerdo con la siguiente fórmula:

$$\text{BCE} = - \frac{1}{N} \sum_{i=1}^{N} \left[ y_i \cdot \log(\hat{y}_i) + (1 - y_i) \cdot \log(1 - \hat{y}_i) \right]$$

En el caso de evaluar una única observación se reduce a:

$$\text{BCE} = -  y \cdot \log(\hat{y}) + (1 - y) \cdot \log(1 - \hat{y}) $$

La derivada de BCE corresponde a: 

$$\frac{\partial \text{BCE}}{\partial \hat{y}} = -\frac{y}{\hat{y}} + \frac{1 - y}{1 - \hat{y}}$$

<b>¿Cómo implementamos BCE en nuestra red Neuronal?</b>

In [ ]:
# Haga acá su implementación de BCE
# Dado que trabajará con la función logaritmo, quizá convenga preveer el casos donde
# log(0) = -infinito, para evitar errores por indefinición matemática.


In [ ]:
# Pruebe si funciona. 
# No asuma que obtendrá de inmediato un resultado mejor que con MSE, ya que logre
# clasificar es un buen resultado. Que sea o no mejor que los resultados previos con 
# MSE dependen de cuanto entrene y qué capacida tenga la red. 

<b>Desvanecimiento del Gradiente (<i>Vanishing Gradient</i>)</b>

El desvanecimiento del gradiente (vanishing gradient) es un problema que ocurre durante el entrenamiento de redes neuronales profundas, especialmente cuando se utilizan ciertas funciones de activación como sigmoidal o tangente hiperbólica. Este fenómeno dificulta el proceso de aprendizaje de la red, ya que afecta la capacidad de las capas más profundas de la red para ajustar sus pesos correctamente.

El ajuste al que se somete cada peso $w$ durante el aprendizaje es proporcional a la derivada del error (función de pérdida) respecto a $\hat{y}$. Ocurre que estamos usando funciones de pérdida como sigmoidal o tangente hiperbólica que tienen tramos extensos donde la derivada del error es cercana a cero, lo cual tiende a reducir la magnitud del ajuste. En las primeras capas no es notorio, aunque sea un ajuste pequeño puede estar compensado por la <i>learning rate</i>, y con suficientes iteraciones se logra igual la optimización. Pero si hay muchas capas en la red, durante el <i>backward propagation</i> el ajuste de pesos se va traspasando de capa en capa hacia atrás, siendo sucesivamente multiplicado por un factor cercado a cero (derivada de función de pérdida), haciendo que el ajuste sea cada vez más pequeño. 

Contribuye a lo anterior la multiplicación de los gradientes, que también van progresivamente haciendose cercanos a cero mientras nos acercamos a un óptimo. Esto significa que capa a capa, hacia atras, se va a comulando la multiplicacion de gradientes pequeños haciendo que la señal de error se desvanesca. Esta componente del problema ya no depende de la forma de la función de activación, y es estructural a la topología de las redes neuronales. 

<br> <img src=https://miro.medium.com/v2/resize:fit:828/format:webp/1*0QPRST83oBicKPE_R4biJA.png
          width="500"/></br>

Resultado de lo anterior, vemos que la magnitud de los ajustes se va haciendo virtualmente cero en las capas iniciales de una red muy profunda (recuerde que el ajuste va hacia atras), y con ello pierde toda capacidad de aprendizaje. 

<br> <img src=https://blog.kakaocdn.net/dn/okTU1/btrCH4Fx7Ro/m5f9UkRkl7CP3dUEgXTtk1/img.png
          width="600"/></br>

In [ ]:
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        # YA NO CONTROLAREMOS ALEATORIEDAD
        #np.random.seed(1234)
        self.weights = np.random.rand(input_size, output_size, ) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        
        # GUARDAMOS LA MAGNITUD DE GRADIENTES PARA ANALIZAR
        self.gradients_magnitude = np.mean(np.abs(weights_error))

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# CREAMOS UNA RED MÁS PROFUNDA, PARA PROPICIAR DESVANECIMIENTO DE GRADIENTE
model.add(FCLayer(entrada_dim, 16))               
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(16, 16))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(16, 16))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(16, 16))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(16, 16)) 
model.add(ActivationLayer(sigmoid, sigmoid_prime))
'''model.add(FCLayer(16, 16)) 
model.add(ActivationLayer(sigmoid, sigmoid_prime))'''
model.add(FCLayer(16, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=10, learning_rate=0.1)

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('\nMATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

In [ ]:
# Después del entrenamiento
gradients = [layer.gradients_magnitude for layer in model.layers if isinstance(layer, FCLayer)]

plt.plot(gradients)
plt.title("Magnitud de los Gradientes a través de las Capas")
plt.xlabel("Capa")
plt.ylabel("Magnitud Media del Gradiente")
plt.show()

Acá podemos apreciar como una red con capacidad "de sobra", al ser demasiado profunda, no logra aprender nada a raíz del desvanecimiento del gradiente. Una solución a este problema es usar funciones de pérdida que mitiguen este problema. 

<b>Agregando más funciones de activación: <i>ReLu</i></b>

Nuestra implementación ya dispone de funciones de activación: Sigmoidal y Tangente Hiperbólica. Quizá recuerde que en ciencia de datos solíamos usar también funciones de activación lineal y lineal rectificada (<i>ReLu</i>), que justamente es popular como mitigador del problema del desvanecimiento del gradiente. 

<b>Implemente usted a continuación <i>ReLu</i> en nuestra red neuronal, y pruebe si mejora la situación previamente descrita. Incluya la gráfica de magnitud de gradiente por capa en su analisis</b>

Para esta tarea tendrá que incluir la derivada de ReLu, con el problema de que justo en $x = 0$ es una función no derivable. Se le sugiere que para ese punto en particular considere que la derivada sea cero, es decir debiera usar una función aproximada de la derivada de ReLu. 

In [ ]:
# Haga acá la implementación de ReLu y su derivada. 
# Considere como abordar la derivada cuando x = 0 y se indefine
# Se sugiere que la ReLu prima de x=0 sea 0. 


In [ ]:
# Pruebe acá una red profunda usando activación ReLu
# Incluya la gráfica de gradientes por capa para analizar
# efecto sobre el desvanecimiento del gradiente. 
# No espere ver que el problema desaparece del todo. 


<b>ReLu y el desvanecimiento del gradiente</b>

Si implementó <i>ReLu</i> y la probó como función de activación de la Red Neuronal (en las capas intermedias), probablemente observe que efectivamente se mitiga el desvanecimiento del gradiente, pero no desaparece y sigue perjudicando el aprendizaje. Sólo una parte del problema se origina en la función de activación. Además, ReLu también tiene un tramo donde su derivada se hace cero. 

ReLU tiene la ventaja de que en la región positiva $(x > 0)$, donde la función es lineal y su derivada es constante e igual a 1. Esto significa que, a diferencia de funciones como sigmoid o tanh, los gradientes no se reducen exponencialmente en cada capa (propiciado por la función de activación), lo que ayuda a que las señales (gradientes) se mantengan lo suficientemente grandes para actualizar los pesos de manera efectiva.

A pesar de que ReLU ayuda a mantener gradientes más grandes que sigmoid o tanh, en redes profundas los gradientes aún pueden disminuir a medida que se propagan hacia atrás a través de muchas capas. Aunque el efecto es menos severo que con otras funciones, sigue siendo un problema a medida que la profundidad de la red aumenta.

<b>ReLu tiende a generar representaciones más dispersas</b>

Otro tema importante con ReLU es que para valores negativos de entrada $(x < 0)$, la salida de ReLU es 0, y la derivada también es 0. Si una neurona recibe una entrada negativa persistente durante el entrenamiento, su salida será 0 y su gradiente será 0, lo que implica que no se actualizará (esta neurona se "muere"). Aunque no causa que el gradiente se desvanezca en el sentido clásico, sí lleva a que algunas neuronas no contribuyan al aprendizaje.

¿Es esto un problema? Veremos que no necesariamente. Esta <b><i>"esparsidad"</i></b> actúa como una forma de regularización porque reduce la capacidad de la red para aprender relaciones espurias o para ajustarse demasiado a los datos de entrenamiento (sobreajuste). Al "apagar" algunas neuronas, ReLU reduce la complejidad efectiva del modelo, lo que puede llevar a una mejor generalización en nuevos datos.

<b>¿Y si incrementamos el número de Neuronas en lugar del número de Capas?</b>

Agregar capas incrementa la capacidad de la red, pero a riesgo de desvanecimiento del gradiente. Podemos evitar ese riesgo incrementando el número de neuronas en lugar de incrementar las capas. Mantengamos algunas capas y muchas más neuronas, veamos como nos va.  

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# USEMOS MÁS NEUROANAS.
# Mantenerlos en potencias de 2 lo hace ver más científico!
model.add(FCLayer(entrada_dim, 256))               
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(256, 256))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(256, 10))                                                   
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=20, learning_rate=0.1) # DUPLIQUEMOS EL ENTRENAMIENTO

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('\nMATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

El éxito de la clasificación sobre el set de testeo no está tan bueno como esperaríamos si observamos cómo se reduce el error durante el entrenamiento, todavía falla del orden del 10% de los casos. 

¿Cómo le va a nuestro modelo en el set de datos de entrenamiento?

In [ ]:
# Usamos el modelo para predecir sobre el conjunto de ENTRENAMIENTO
y_hat = model.predict(X_train)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_train_value, y_hat)

print('\nMATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de ENTRENAMIENTO del modelo ANN es: {:.3f}'.format(accuracy_score(y_train_value,y_hat)))

Claramente en el entrenamiento se logra mucho mejor resultado con los datos del entrenamiento que de testeo. Esto ya sabemos de qué se trata. 

<br> <img src=https://pbs.twimg.com/media/FSAM8F1WQAMIIwB.jpg 
          width="250">
                    
<b>OVERFITTING, viejo enemigo</b>

Aumentar la capacidad de representación de la red, ya sea por disponer de más capas, más neuronas o más tiempo de entrenamiento, estamos generando las condiciones que facilitan el <b><i>overfitting</i></b>. Podemos tratar de controlarlo por la vía de seleccionar hiperparámetros de configuración del modelo buscando que no tenga demasiada capacidad. Sin embargo, con esto estamos limitando nuestra posibilidad de aprovechar modelos más robustos. El aprendizaje profundo, que ha permitido generar los modelos AI que sorprenden al mundo, requiere que podamos enfrentar este problema por otra vía.

<b>REGULARIZACIÓN</b>

La regularización es un conjunto de técnicas utilizadas en machine learning y en particularmente en redes neuronales, para prevenir el overfitting, es decir que el modelo se ajuste demasiado a los datos de entrenamiento y pierde capacidad de generalizar sobre nuevas observaciones. Por lo general consiste en adoptar estrategias que reduzcan la capacidad del modelo, evitando así que se sobreajuste. 

Existe distintas estrategias de regularización, a continuación revisamos algunas de las más frecuentes:

<b>Regularización L1 (Lasso)</b>: Añade a la función de costo un término penalizador proporcional al valor absoluto de los pesos. Así, redes que entrenan pesos muy grandes marcarán peor resultado que otras más simples. Esto puede llevar a que algunos pesos se vuelvan exactamente cero, lo que resulta en un modelo más <i>esparso</i> (disperso, muchos de los pesos son cero).

<b>Regularización L2 (Ridge)</b>: Añade un término penalizador proporcional al cuadrado de la magnitud de los pesos. Es la forma más común de regularización y tiende a reducir todos los pesos hacia valores pequeños, pero no necesariamente cero.

<br> <img src=https://sebastianraschka.com/images/faq/reqularization-linear/l2.png
          width="350"/>

<b>Dropout</b>: Durante el entrenamiento, dropout apaga aleatoriamente una proporción de neuronas en las capas específicas, lo que evita que cualquier neurona dependa demasiado de las salidas de otras neuronas. Esencialmente, al hacer esto, estás entrenando una versión "diluida" de tu red en cada paso, y el modelo final es una especie de promedio de todas estas redes más pequeñas. Esto ayuda a mejorar la robustez y generalización del modelo.

<br> <img src=https://miro.medium.com/v2/resize:fit:1044/1*iWQzxhVlvadk6VAJjsgXgg.png
          width="400"/>

<b>Early stopping</b>: En lugar de entrenar la red hasta que el error de entrenamiento sea mínimo, se detiene el entrenamiento una vez que el error en un conjunto de validación separado comienza a aumentar. Esto significa que la red se ha ajustado lo suficiente a los datos de entrenamiento, pero antes de que comience a sobreajustarse.

<br> <img src=https://www.researchgate.net/profile/Ramazan-Gencay/publication/3302948/figure/fig1/AS:671509056073733@1537111643349/Early-stopping-based-on-cross-validation.pbm
          width="250"/>

<b>Noise regularization</b>: Se añade ruido a las entradas o a las activaciones dentro de la red durante el entrenamiento, lo que puede ayudar a evitar el sobreajuste al hacer que la red sea más robusta a pequeñas perturbaciones en sus entradas.

<b>Data augmentation</b>: Aumentar tus datos significa crear versiones modificadas de los datos de entrada para ampliar el conjunto de entrenamiento. En el caso de imágenes puede ser rotarlas, recortarlas o voltearlas para crear más ejemplos de entrenamiento. 

Cada una de estas estrategias de regularización se puede usar individualmente, así como combinar dos o más de estas. Como siempre ocurre en nuestra disciplina, habrá que probar y buscar entre combinaciones y parámetros, lo cual suele ser un proceso largo, que nos lleve a identificar qué funciona mejor con el problema que estemos atacando. 

<b>Implementemos regularización $L2$ sobre nuestra red neuronal</b> 
<br> <img src=https://i.pinimg.com/736x/a9/a2/45/a9a245e8b6eea005944fc4e5dec7ee79.jpg
          width="300"/>

Basta con sumar un coeficiente $\lambda*\|\mathbf{W}\|_2$ a la función de costos. Sin embargo, necesitaríamos acceder a cada peso de la red al momento de computar el costo, lo cual haría bastante menos eficiente la implementación. Podemos usar un truco para incorporar la regularización $L2$ directamente en el paso de backpropagation, añadiendo la derivada del término de penalización ($L2$) con respecto a los pesos. La derivada de $ \lambda \sum_i w_i^2 $ con respecto a $ w_i $ es simplemente $ 2\lambda w_i $. Es decir, la regularización $L2$ es proporcional a $ \lambda $ en la transmisión del error en la fase de backpropagation, así que eso es lo que implementaremos, evitando así cálculos adicionales con los pesos $w$ al momento de computar la pérdida. 
 
Note que esta solución no afecta el cálculo que ya tenemos de la función de costos, sólo produce el efecto de si hubieramos incluido $\lambda*\|\mathbf{W}\|_2$ al momento de hacer el cálculo del gradiente para efecto del ajuste de pesos. Por otro lado, eso es lo único que necesitamos realmente, no nos importa que la función de pérdida (costo) se siga calculando sin la regularización para efectos de reporte. 

In [ ]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size, lambda_reg=0):
        #np.random.seed(1234)
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.lambda_reg = lambda_reg  # Se agrega parámetro coeficiente lambda de regularización L2

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # Adicionamos acá el término de regularización L2 que castiga error en los pesos
        weights_error += self.lambda_reg * self.weights

        # Actualizar los parámetros
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

Con este cambio el modelo castigará soluciones en las que los pesos tienen valores grandes, lo que la lleva a buscar que los pesos sean del menor tamaño posible. Usar $L2$ implica que el castigo es proporcional al cuadrado de los pesos, lo cual inhibe a que el modelo reduzca la suma total de pesos por la vía de desactivar neuronas (dejar pesos en cero) para compensar pesos grandes. Esto permite que todos los pesos de la red sigan siendo distintos a cero, pero lo más cercanos posible. 

Volvemos a definir y entrenar la red, pero esta vez usando un parámetro de regularización $L2$ en la forma de un coeficiente $\lambda$. 

In [ ]:
# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Definimos el término de regularización lambda. 
lambda_reg=0.001

# REPETIMOS EL MISMO EXPERIMENTO, PERO CON REGULARIZACIÓN
model.add(FCLayer(entrada_dim, 256, lambda_reg=lambda_reg))  # AGREGAREMOS PARÁMETRO L2 A CADA CAPA DENSA            
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(256, 256, lambda_reg=lambda_reg))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(256, 10, lambda_reg=lambda_reg))  
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Usar el modelo creado
model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=20, learning_rate=0.1) # DUPLIQUEMOS EL ENTRENAMIENTO

# Usamos el modelo para predecir sobre el conjunto de prueba
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('\nMATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

# Usamos el modelo para predecir sobre el conjunto de ENTRENAMIENTO
y_hat = model.predict(X_train)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_train_value, y_hat)

print('\nMATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de ENTRENAMIENTO del modelo ANN es: {:.3f}'.format(accuracy_score(y_train_value,y_hat)))

Usando Regularización L2 una Red con alta capacidad de representación, permite regular su capacidad de aprender con demasiada precisión los datos de entrenamiento, lo que en teoría nos debiera permitir aprovechar las capacidad de la red para obtener mejor generalización. 

El coeficiente $\lambda$ pasa a ser un nuevo hiperparámetro del modelo, así que se debiera realizar una búsqueda exhaustiva sobre todos los hiperparámetros, mediante un GridSearch y Validación Cruzada, para encontrar un mejor modelo. Seguramente el resultado que está a la vista todavía se puede mejorar con los recursos que hemos visto hasta acá, pero el proceso de búsqueda será largo.

<b>¿Cómo podemos acelerar el entrenamiento?</b>

Nuestra implementación básica de red neuronal realiza el ajuste de sus parámetros internos de a una observación cada vez. Una forma de acelerar el proceso sería implementar <b><i>mini batch</i></b>. 

<b>MINI-BATCH</b>

Cuando usando <i>Gradient Descent</i> ajustamos los pesos con muestra individual en el conjunto de datos de entrenamiento, se le conoce como <i>Stochastic Gradient Descent</i>. Si en lugar de con cada observación en la muestra usamos todo el lote del conjunto de entrenamiento, al final de cada <i>epoch</i>, se denomina <i>Bath Gradient Descent</i>. Esta última técnica reduce el tiempo de ejecución del algoritmo, a costa de hacer más lenta la convergencia del algoritmo. 

<i>Mini-Batch</i> equilibria ambos enfoques. En lugar de calcular el gradiente utilizando todo el conjunto de datos (como en BGD) o solo una muestra (como en SGD), se calcula utilizando un pequeño subconjunto de datos llamado mini-batch. Este subconjunto de datos puede tener un tamaño arbitrario, pero se recomienda usar potencias de dos: 16, 32, 64, 128, etc. 

Si implementamos mini-batch, podremos hacer búsquedas más rápidas de hiperparámetros adecuados a cada problema. 

In [ ]:
# TAREA PARA LA CASA
# Haga acá o en su propia implementación en GitHub una versión que use MINI BATCH.

# Debe permitir configurar como hiperparámetro el tamaño del mini batch al hacer fit.
# El último mini lote tendrá que tener un tamaño de ajuste, para calzar con el lote completo.
# Su implementación debe enfocarse en usar el mini batch en la etapa de backward propagation
# Obtenga el gradiente promedio del mini lote, y realice el ajuste de pesos una vez por mini lote.
# Se sugiere realizar barajado aleatorio del mini lote (sin perder de vista orden de etiquetas).

#... próxima clase vengan preparados para comentar cómo les fue. 


<b> CONCLUSIÓN </b>

Hemos probado que nuestra implementación de Red Neuronal puede crecer, incorporando nuevas funciones de activación y de pérdida. 

Hemos ilustrado el problema del desvanecimiento del gradiente en redes con muchas capas, que impide el aprendizaje de la red. También hemos podido apreciar el efecto del sobreajuste en la capacidad de generalizar de nuestras redes. 

Hemos incorporando Regularización L2 (Ridge) para comenzar a combatir el sobreajuste. Queda pendiente implementar otras estrategias de regularización. Disponer de herramientas para hacer regularización será importante, en la medida que adoptemos modelos más potentes para resolver problemas específicos, cosa que esperamos hacer desde la próxima clase. Así que no es mala idea ir pensando en como incluiría otras técnicas de regularización, sean las mencionadas en este apunte como otras que pueda investigar. 

Si bien estamos ya reconociendo imágenes con esta implementación, aún simple, de red neuronal, necesitamos ir más allá. En la próxima clase nos enfocaremos en realizar cambios a la topología de la red, a fin de hacerla más propicia para tareas que involucran imágenes.  